In [2]:
import sys

# %pip install pandas
import pandas as pd

# %pip install numpy
import numpy as np

# %pip install textblob
from textblob import TextBlob

# %pip install nltk
import nltk

In [2]:
reviews_df = pd.read_csv('Restaurant_Reviews.tsv', sep='\t')
print(reviews_df)

                                                Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]


In [4]:
review_column = reviews_df["Review"]
liked_column = reviews_df["Liked"]
sentiment_polarity = []

for column in review_column:
    blob = TextBlob(column)
    sentiment_polarity.append(blob.sentiment.polarity)

pearson_corr_coeff = np.corrcoef(sentiment_polarity, liked_column)
print(pearson_corr_coeff)

[[1.         0.59005924]
 [0.59005924 1.        ]]
